In [4]:
import pandas as pd

data = {
    'machine_id': [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2],
    'process_id': [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
    'activity_type': ['start', 'end', 'start', 'end', 'start', 'end', 'start', 'end', 'start', 'end', 'start', 'end'],
    'timestamp': [0.712, 1.520, 3.140, 4.120, 0.550, 1.550, 0.430, 1.420, 4.100, 4.512, 2.500, 5.000]
}

df = pd.DataFrame(data)

print(df)

    machine_id  process_id activity_type  timestamp
0            0           0         start      0.712
1            0           0           end      1.520
2            0           1         start      3.140
3            0           1           end      4.120
4            1           0         start      0.550
5            1           0           end      1.550
6            1           1         start      0.430
7            1           1           end      1.420
8            2           0         start      4.100
9            2           0           end      4.512
10           2           1         start      2.500
11           2           1           end      5.000


In [6]:
df = pd.DataFrame(data)

# Calculate processing time for each process
df['processing_time'] = df.groupby(['machine_id', 'process_id'])['timestamp'].diff()

print(df)

# Drop rows where activity_type is 'start' (to keep only end timestamps)
df = df[df['activity_type'] == 'end']

print("\n", df)

# Group by machine_id and calculate average processing time
result = df.groupby('machine_id')['processing_time'].mean().reset_index()

# Round processing time to 3 decimal places
result['processing_time'] = result['processing_time'].round(3)

print("\n", result)

    machine_id  process_id activity_type  timestamp  processing_time
0            0           0         start      0.712              NaN
1            0           0           end      1.520            0.808
2            0           1         start      3.140              NaN
3            0           1           end      4.120            0.980
4            1           0         start      0.550              NaN
5            1           0           end      1.550            1.000
6            1           1         start      0.430              NaN
7            1           1           end      1.420            0.990
8            2           0         start      4.100              NaN
9            2           0           end      4.512            0.412
10           2           1         start      2.500              NaN
11           2           1           end      5.000            2.500

     machine_id  process_id activity_type  timestamp  processing_time
1            0           0      

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CalculateAverageProcessingTime") \
    .getOrCreate()

# Sample data
data = {
    'machine_id': [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2],
    'process_id': [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
    'activity_type': ['start', 'end', 'start', 'end', 'start', 'end', 'start', 'end', 'start', 'end', 'start', 'end'],
    'timestamp': [0.712, 1.520, 3.140, 4.120, 0.550, 1.550, 0.430, 1.420, 4.100, 4.512, 2.500, 5.000]
}

# Create DataFrame
df = spark.createDataFrame(pd.DataFrame(data))

# Calculate processing time for each process
window_spec = Window.partitionBy('machine_id', 'process_id').orderBy('timestamp')
df = df.withColumn('processing_time', 
                   (F.col('timestamp') - 
                    F.lag('timestamp', default=0).over(window_spec)))

# Filter out 'start' activities
df = df.filter(df.activity_type == 'end')

result = df.groupBy('machine_id').agg(F.round(F.avg('processing_time'), 3).alias('processing_time'))

result.show()

NameError: name 'pd' is not defined

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CalculateAverageProcessingTime") \
    .getOrCreate()

# Sample data
data = {
    'machine_id': [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2],
    'process_id': [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
    'activity_type': ['start', 'end', 'start', 'end', 'start', 'end', 'start', 'end', 'start', 'end', 'start', 'end'],
    'timestamp': [0.712, 1.520, 3.140, 4.120, 0.550, 1.550, 0.430, 1.420, 4.100, 4.512, 2.500, 5.000]
}

# Create DataFrame
df = spark.createDataFrame(pd.DataFrame(data))

# Create a temporary view
df.createOrReplaceTempView("activity_table")

r1 = spark.sql("""select * from activity_table""")
r1.show()

# SQL query to calculate processing time and average processing time
result = spark.sql("""
    SELECT 
        machine_id,
        round(AVG(processing_time), 3) AS processing_time
    FROM (
        SELECT 
            machine_id,
            process_id,
            MAX(timestamp) - MIN(timestamp) AS processing_time
        FROM 
            activity_table
        GROUP BY
            machine_id, process_id
    ) t
    GROUP BY 
        machine_id
""")

# Show the result
result.show()


In [ ]:
# Write your MySQL query statement below
WITH ProcessTimes AS
    SELECT 
        machine_id,
        process_id,
        MAX(timestamp) - MIN(timestamp) AS processing_time
    FROM 
        Activity
    GROUP BY
        machine_id, process_id
)
SELECT 
    machine_id,
    round(AVG(processing_time), 3) AS processing_time
FROM 
    ProcessTimes
GROUP BY 
    machine_id;

In [ ]:
select a.machine_id, a.process_id, a.activity_type, b.activity_type, a.timestamp, b.timestamp
from Activity a
join Activity b
on a.machine_id = b.machine_id and a.process_id = b.process_id and a.activity_type='start' and b.activity_type='end'


select a.machine_id, round(avg((b.timestamp - a.timestamp)), 3) as processing_time
from Activity a
join Activity b
on a.machine_id = b.machine_id and a.process_id = b.process_id and a.activity_type='start' and b.activity_type='end'
group by a.machine_id

In [ ]:
SELECT a1.machine_id,
    ROUND(SUM(a2.timestamp-a1.timestamp)/COUNT(distinct a1.process_id),3)AS processing_time
FROM Activity a1
JOIN Activity a2 ON a1.machine_id=a2.machine_id AND a1.process_id=a2.process_id AND a1.activity_type!=a2.activity_type AND a1.timestamp < a2.timestamp
GROUP BY machine_id